# compute Euclidean Distance
Euclidean Distance is a termbase in mathematics; therefore I won’t discuss it at length. Generally speaking, it is a straight-line distance between two points in Euclidean Space. The formula for euclidean distance for two vectors $ u, v ∈ R^n $ is: <br>
$$ u=\begin{bmatrix} u_{1} & u_{2} & u_{3} & \vdots  & u_{n}  \end{bmatrix} $$      
$$ v=\begin{bmatrix} v_{1} & v_{2} & v_{3} & \vdots  & v_{n}  \end{bmatrix} $$

$$ d(v,u)= \sqrt{ \sum_{i=1}^{n}|v_i-u_i|^2 } $$


how to write Matrix ?
\begin{bmatrix}
    x_{11} & x_{12} & x_{13} & \dots  & x_{1n} \\
    x_{21} & x_{22} & x_{23} & \dots  & x_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{d1} & x_{d2} & x_{d3} & \dots  & x_{dn}
\end{bmatrix}

## Einsum
https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html <br>
https://semantive.com/high-performance-computation-in-python-numpy-2/  <br>

Einstein Summation Convention is an elegant way to express a common operation on matrices like a dot product, a sum over indices and a matrix transposition. At first, it may look impractical due to the complex syntax, but it will turn out that its implementation is very efficient.

![title](https://semantive.com/wp-content/uploads/2019/02/performance_2.png)

## numpy.linalg.norm
https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html

## scipy.spatial.distance
https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


In [ ]:
import numpy as np

def dummy_mat(mat_v, mat_u):
   return [sum((v_i - u_i)**2 for v_i, u_i in zip(v, u))**0.5 for v, u in zip(mat_v, mat_u)]

def bare_numpy_mat(mat_v, mat_u):
   return np.sqrt(np.sum((mat_v - mat_u) ** 2, axis=1))

def l2_norm_mat(mat_v, mat_u):
   return np.linalg.norm(mat_v - mat_u, axis=1)

def scipy_distance_mat(mat_v, mat_u):
   # Unfortunately, the scipy_distance only works on 1D-arrays, so we are not able to vectorize it again.
   return list(map(distance.euclidean, mat_v, mat_u))

def einsum_mat(mat_v, mat_u):
   mat_z = mat_v - mat_u
   return np.sqrt(np.einsum('ij,ij->i', mat_z, mat_z))

In [ ]:
# closest point
def closest_node(node, nodes):
    node = np.asarray(node);  nodes = np.asarray(nodes)
    d = nodes - node                                  # Nx3 array of Rij vectors
    dist2 = np.sqrt(np.einsum('ij,ij->i', d, d))      # Nx1 array of distances
    return np.argmin(dist2)

In [11]:
# distance from node to nodes
def dist2_node2nodes(node, nodes):
    node = np.asarray(node);  nodes = np.asarray(nodes)
    d = nodes - node                                  # Nx3 array of Rij vectors
    dist2 = np.sqrt(np.einsum('ij,ij->i', d, d))      # Nx1 array of distances
    return dist2, d

## test performance

In [17]:
import numpy as np
a = np.array([1,1,1])
b = np.array([[1,1,1], [2,2,2], [-3,-3,-3]])
print(b-a)

[[ 0  0  0]
 [ 1  1  1]
 [-4 -4 -4]]


In [61]:
import numpy as np
a = np.array([1,1,1])
b = np.arange(3000).reshape(1000,3)
d=b-a

In [65]:
%%timeit
# use bare numpy
d1 = np.sqrt((d*d).sum(axis=1))

17.9 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [66]:
%%timeit
# use einsumn
d2 = np.sqrt(np.einsum('ij,ij->i', d, d))

11 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [67]:
%%timeit
# dummy use
d3 = np.sqrt(d[:,0]**2 + d[:,1]**2 + d[:,2]**2)

12.4 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [71]:
import numpy as np
a = np.array([1,1,1])
b = np.arange(30).reshape(10,3)
d=b-a

d1 = np.sqrt((d*d).sum(axis=1))
d2 = np.sqrt(np.einsum('ij,ij->i', d, d))
d3 = np.sqrt(d[:,0]**2 + d[:,1]**2 + d[:,2]**2)

print(d1)
print(d2)
print(d3)

[ 1.41421356  5.38516481 10.48808848 15.65247584 20.83266666 26.01922366
 31.20897307 36.40054945 41.59326869 46.78675026]
[ 1.41421356  5.38516481 10.48808848 15.65247584 20.83266666 26.01922366
 31.20897307 36.40054945 41.59326869 46.78675026]
[ 1.41421356  5.38516481 10.48808848 15.65247584 20.83266666 26.01922366
 31.20897307 36.40054945 41.59326869 46.78675026]


# 2. Dot product

https://stackoverflow.com/questions/26089893/understanding-numpys-einsum

### What does einsum do?
Imagine that we have two multi-dimensional arrays, A and B. Now let's suppose we want to...

multiply A with B in a particular way to create new array of products; and then maybe
sum this new array along particular axes; and then maybe
transpose the axes of the new array in a particular order.
There's a good chance that einsum will help us do this faster and more memory-efficiently that combinations of the NumPy functions like multiply, sum and transpose will allow.

### How does einsum work?
Here's a simple (but not completely trivial) example. Take the following two arrays:

In [16]:
A = np.array([0, 1, 2])
B = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])

We will multiply A and B element-wise and then sum along the rows of the new array. In "normal" NumPy we'd write:

In [18]:
d = A[:, np.newaxis] * B
c = d.sum(axis=1)
print(d)
print(c)

[[ 0  0  0  0]
 [ 4  5  6  7]
 [16 18 20 22]]
[ 0 22 76]


So here, the indexing operation on A lines up the first axes of the two arrays so that the multiplication can be broadcast. 
The rows of the array of products is then summed to return the answer.

Now if we wanted to use einsum instead, we could write:

In [19]:
np.einsum('i,ij->i', A, B)

array([ 0, 22, 76])

### Here is what happens next:

* A has one axis; we've labelled it i. And B has two axes; we've labelled axis 0 as i and axis 1 as j.
* By repeating the label i in both input arrays, we are telling einsum that these two axes should be multiplied together. In other words, we're multiplying array A with each column of array B, just like A[:, np.newaxis] * B does.
* Notice that j does not appear as a label in our desired output; we've just used i (we want to end up with a 1D array). By omitting the label, we're telling einsum to sum along this axis. In other words, we're summing the rows of the products, just like .sum(axis=1) does.

That's basically all you need to know to use einsum. It helps to play about a little; if we leave both labels in the output, 'i,ij->ij', we get back a 2D array of products (same as A[:, np.newaxis] * B). If we say no output labels, 'i,ij->, we get back a single number (same as doing (A[:, np.newaxis] * B).sum()).

The great thing about einsum however, is that is does not build a temporary array of products first; it just sums the products as it goes. This can lead to big savings in memory use.

https://stackoverflow.com/questions/26089893/understanding-numpys-einsum

## a. dot product between a vector and a list of vectors

In [56]:
import numpy as np
a = np.array([1,1,1])
b = np.array([[1,1,1], [2,2,2], [-3,-3,-3], [-3,-3,-3]])

In [21]:
# dot product a vector and a list of vectors
dot = b @ a    # can not use dot = a @ b
print(dot)

[ 3  6 -9 -9]


In [59]:
# use einsum
dot = np.einsum('ij,j->i', b, a)
print(dot)

[ 3  6 -9 -9]


In [60]:
%%timeit
dot = np.einsum('ij,j->i', b, a)

1.19 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [61]:
%%timeit
dot = b @ a 

751 ns ± 2.51 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## b. dot product between 2 vector 
equivalent to element-wise multiply 2 vectors a*b
$$ d = \sum_{i=1}^{n}{a_i.b_i} = a_ib_i $$
* a*b = np.einsum('i,i->i',a,b)

In [13]:
import numpy as np
a = np.array([1,1,1,2,3])
b = np.array([2,3,4,5,6])

print(a*b)
print(np.einsum('i,i->i',a,b))

[ 2  3  4 10 18]
[ 2  3  4 10 18]


In [11]:
%%timeit
a*b

349 ns ± 8.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit
np.einsum('i,i->i',a,b)

1.16 µs ± 7.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
np.ones(a.shape[0])

array([1., 1., 1., 1., 1.])

## c. element-wise multiply 3 vectors
a*b*c = np.einsum('i,i,i->i',a,b,c)

In [18]:
import numpy as np
a = np.array([1,1,1,2,3])
b = np.array([2,3,4,5,6])
c = np.array([2,3,4,2,2])

d = np.einsum('i,i,i->i',a,b,c)
print(d)
print(a*b*c)

[ 4  9 16 20 36]
[ 4  9 16 20 36]


In [21]:
%%timeit
a*b*c

706 ns ± 102 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
%%timeit
np.einsum('i,i,i->i',a,b,c)

1.72 µs ± 6.89 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## d. multiply 2 matrix (rotation matrix)
np.einsum('ij,jk', a, b) describes traditional matrix multiplication and is equivalent to np.matmul(a,b).

In [3]:
import numpy as np
a = np.array([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [4,5,5]])
R = np.array([[1,2,3], [0,3,4], [4,5,6]])
print(np.matmul(a,R))
print(np.einsum('ij,jk', a, R))

[[ 5 10 13]
 [10 20 26]
 [15 30 39]
 [20 40 52]
 [24 48 62]]
[[ 5 10 13]
 [10 20 26]
 [15 30 39]
 [20 40 52]
 [24 48 62]]


## 5. multiply 1 matrix with each row of another matrix (3D Rotation)

In [19]:
import numpy as np
a = np.array([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [4,5,5]])
R = np.array([[1,2,3], [0,3,4], [4,5,6]])
print(np.matmul(a,R))
print(np.einsum('ij,jk', a, R))

c = np.zeros((a.shape[0],3))
for i in range(a.shape[0]):
    c[i,:] = np.matmul(R, a[i,:])
    
print(c)
print(np.einsum('ik,jk', a, R))      # multiply matrix R with each row of a

[[ 5 10 13]
 [10 20 26]
 [15 30 39]
 [20 40 52]
 [24 48 62]]
[[ 5 10 13]
 [10 20 26]
 [15 30 39]
 [20 40 52]
 [24 48 62]]
[[ 6.  7. 15.]
 [12. 14. 30.]
 [18. 21. 45.]
 [24. 28. 60.]
 [29. 35. 71.]]
[[ 6  7 15]
 [12 14 30]
 [18 21 45]
 [24 28 60]
 [29 35 71]]


In [2]:
import numpy as np
a = np.arange(5)
print(a)

[0 1 2 3 4]


In [8]:
a[0:]

array([0, 1, 2, 3, 4])